In [1]:
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output,callback

from scipy.signal import butter, lfilter, freqz

import numpy as np
import pandas as pd

In [2]:
def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [3]:
app = Dash(__name__)
app.layout = html.Div([
    dcc.Slider(
                id='filter-order',
                min = 2.0 ,
                max = 8.0 ,
                step = 1,
                value = 2,
                marks = {i:str(i) for i in range(0,9)},
                updatemode='drag',
            ),
     dcc.Slider(
                id='cutoff',
                min = 2 ,
                max = 270 ,
                step = 1,
                value = 2,
                marks = {i:str(i) for i in range(0,270,10)},
                updatemode='drag',
            ),
    dcc.Slider(
                id='max',
                min = 0.6 ,
                max = 1 ,
                step = 0.1,
                value = 1,
                updatemode='drag',
            ),
    dcc.Slider(
                id='min',
                min = 0.0 ,
                max = 0.5 ,
                step = 0.05,
                value =  0,
                updatemode='drag',
            ),
    html.Div(children = [dcc.Graph(
        id='filter',style={'display':'inline-block'}),
        dcc.Graph(
        id='heatmap',style={'display':'inline-block'})])
    ])

@callback(
    [Output(component_id='filter', component_property='figure'),
    Output(component_id='heatmap', component_property='figure')],
    [Input(component_id='filter-order', component_property='value'),
     Input(component_id='cutoff', component_property='value'),
    Input(component_id='max', component_property='value'),
    Input(component_id='min', component_property='value')]
)
def update_filter(order, cutoff, max_val,min_val):
    
    cutoff = cutoff/10
    fs = 54
    b, a = butter_lowpass(cutoff, fs, order)
    w, h = freqz(b, a, fs=fs, worN=270)
    y = np.abs(h)
    
    max_rssi = np.full((1, 270, 24), max_val)
    min_rssi = np.full((1, 270, 24), min_val)
       
    y = y * (max_val - min_val) + min_val
    
    fig = go.Figure([go.Scatter(x=[i for i in range(y.shape[0])], y=y)])
    '''
    max_rssi = np.full((1, 270, 24), max_val)
    min_rssi = np.full((1, 270, 24), min_val)
    '''   
    #y = y * (max_val - min_val) + min_val
    y = y.reshape(270,1)
    hrs = np.ones((1,24))
    img = (y*hrs)
    
    img = np.reshape(img,(1,270,24))
    images = np.r_[img, max_rssi, min_rssi]
    
    heat_map = px.imshow(images,
                zmin=0,
                zmax=1,
                facet_col=0,
                aspect = 'auto',
                color_continuous_scale='jet')

    heat_map.update_layout(margin = dict(t=70, l=50, r=0, b=5),
                          coloraxis_showscale = False,
                          height = 400,
                          width = 600)
     
    return fig , heat_map

if __name__ == '__main__':
    app.run_server(mode = 'jupyterlab')